In [25]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from IPython.display import SVG, Image
from livelossplot.inputs.tf_keras import PlotLossesCallback
import tensorflow as tf
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
# calculates f1 for 1:100 dataset with 95tp, 5fn, 55fp
from sklearn.metrics import f1_score
print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.6.0


In [26]:
img_size = 48
batch_size = 64
datagen_train = ImageDataGenerator(horizontal_flip=True)
train_generator = datagen_train.flow_from_directory("../../content",
                                                    target_size=(img_size,img_size),
                                                    color_mode="grayscale",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)
train_generator


Found 0 images belonging to 3 classes.


In [27]:
# Initialising the CNN
model = Sequential()
# 1 - Convolution
model.add(Conv2D(64,(3,3), padding='same', input_shape=(40, 30, 9)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# 2nd Convolution layer
model.add(Conv2D(128,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# 3rd Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# 4th Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# Flattening
model.add(Flatten())
# Fully connected layer 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(1, activation='softmax'))
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-1,
        decay_steps=10000,
        decay_rate=0.2)
opt = Adam(learning_rate = lr_schedule)
model.compile(loss='mean_squared_error',  optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 40, 30, 64)        5248      
_________________________________________________________________
batch_normalization_12 (Batc (None, 40, 30, 64)        256       
_________________________________________________________________
activation_12 (Activation)   (None, 40, 30, 64)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 20, 15, 64)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 20, 15, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 20, 15, 128)       204928    
_________________________________________________________________
batch_normalization_13 (Batc (None, 20, 15, 128)      

In [28]:
from PIL import Image
from numpy import asarray
import numpy as np
import os
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os
from keras.utils import np_utils

In [29]:
def readImage(filePath):
    img = Image.open(filePath)
    
    size=(30,40)
    #resize image
    out = img.resize(size)

    # asarray() class is used to convert
    # PIL images into NumPy arrays
    numpydata = asarray(out)
    numpydata = np.repeat(numpydata[:, :, np.newaxis], 3, axis=2)
    # <class 'numpy.ndarray'>
    #print(type(numpydata))

    #  shape
    #print(numpydata.shape)
    return numpydata
print(readImage('data/nervus/ISIC_0001769.jpg'))

[[[[142 112  87]
   [142 112  87]
   [142 112  87]]

  [[157 121  91]
   [157 121  91]
   [157 121  91]]

  [[168 130  98]
   [168 130  98]
   [168 130  98]]

  ...

  [[174 139 113]
   [174 139 113]
   [174 139 113]]

  [[169 135 110]
   [169 135 110]
   [169 135 110]]

  [[159 130 108]
   [159 130 108]
   [159 130 108]]]


 [[[148 115  89]
   [148 115  89]
   [148 115  89]]

  [[161 123  95]
   [161 123  95]
   [161 123  95]]

  [[170 133 103]
   [170 133 103]
   [170 133 103]]

  ...

  [[175 139 114]
   [175 139 114]
   [175 139 114]]

  [[171 136 112]
   [171 136 112]
   [171 136 112]]

  [[164 132 111]
   [164 132 111]
   [164 132 111]]]


 [[[155 120  95]
   [155 120  95]
   [155 120  95]]

  [[165 129 102]
   [165 129 102]
   [165 129 102]]

  [[170 133 103]
   [170 133 103]
   [170 133 103]]

  ...

  [[178 144 120]
   [178 144 120]
   [178 144 120]]

  [[173 140 116]
   [173 140 116]
   [173 140 116]]

  [[168 136 115]
   [168 136 115]
   [168 136 115]]]


 ...


 [[[154 129 

In [30]:
listFiles = os.listdir('data/melanoma/')
print(listFiles[:5])

['ISIC_0012099.jpg', 'ISIC_0012151.jpg', 'ISIC_0012288.jpg', 'ISIC_0012434.jpg', 'ISIC_0013232.jpg']


In [31]:
#load Benign
listCancer = []
listResults = []
for elem in listFiles:
    img = readImage('data/melanoma/'+elem)
    listCancer += [img]
    listResults += [np.array([1])]
print(listCancer[:3])
print(img.shape)

[array([[[[ 54,  55,  58],
         [ 54,  55,  58],
         [ 54,  55,  58]],

        [[168, 168, 169],
         [168, 168, 169],
         [168, 168, 169]],

        [[184, 184, 185],
         [184, 184, 185],
         [184, 184, 185]],

        ...,

        [[190, 180, 183],
         [190, 180, 183],
         [190, 180, 183]],

        [[172, 164, 169],
         [172, 164, 169],
         [172, 164, 169]],

        [[ 66,  72,  89],
         [ 66,  72,  89],
         [ 66,  72,  89]]],


       [[[ 99,  99, 102],
         [ 99,  99, 102],
         [ 99,  99, 102]],

        [[179, 178, 179],
         [179, 178, 179],
         [179, 178, 179]],

        [[185, 183, 184],
         [185, 183, 184],
         [185, 183, 184]],

        ...,

        [[192, 180, 184],
         [192, 180, 184],
         [192, 180, 184]],

        [[184, 172, 175],
         [184, 172, 175],
         [184, 172, 175]],

        [[100,  97, 108],
         [100,  97, 108],
         [100,  97, 108]]],


       

In [32]:
listFiles = os.listdir('data/nervus/')
print(listFiles[:5])
for elem in listFiles:
    img = readImage('data/nervus/'+elem)
    listResults += [np.array([0])]
    listCancer += [img]

['ISIC_0001769.jpg', 'ISIC_0001852.jpg', 'ISIC_0001871.jpg', 'ISIC_0003462.jpg', 'ISIC_0003539.jpg']


In [33]:
listFiles = os.listdir('data/seborrheic_keratosis/')
print(listFiles[:5])
for elem in listFiles:
    img = readImage('data/seborrheic_keratosis/'+elem)
    listResults += [np.array([2])]
    listCancer += [img]

['ISIC_0012143.jpg', 'ISIC_0012204.jpg', 'ISIC_0012210.jpg', 'ISIC_0012254.jpg', 'ISIC_0012380.jpg']


In [10]:
from imblearn.over_sampling import SMOTE
print(len(listCancer))
print(len(listResults))
oversample = SMOTE()
X, y = oversample.fit_resample(np.array(listCancer).reshape(len(listCancer),-1), listResults)
print(len(X))
print(len(y))
listCancer = list(X)
listResults = list(y)
print(listResults)

150
150
234
234
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [38]:
x_train=[]
y_train=[]
accs=[]
def main(pz):
    model.compile(loss='mean_squared_error',  optimizer=opt, metrics=['accuracy'])
    per = np.random.permutation(len(listCancer))
    ln = int(len(listCancer) * 0.8)
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    positions = []
    for i in range(ln):
        positions += [per[i]]
   
    for i in range(len(listCancer)):
        if i in positions:
            x_train += [listCancer[i]]
            y_train += [listResults[i]]
        else:
            x_test += [listCancer[i]]
            y_test += [listResults[i]]
            
    ln = len(x_test) // 2
    
    x_valid = x_test[ln:]
    y_valid = y_test[ln:]
    
    x_test = x_test[:ln]
    y_test = y_test[:ln]
    
    x_train = np.array(x_train)
    x_train = x_train.reshape(len(x_train), 40, 30, 9)
    y_train = np.array(y_train)
    

    x_test = np.array(x_test)
    x_test = x_test.reshape(len(x_test), 40, 30, 9)
    y_test = np.array(y_test)
    
    x_valid = np.array(x_valid)
    x_valid = x_valid.reshape(len(x_valid), 40, 30, 9)
    y_valid = np.array(y_valid)
    
    model.fit(x_train, y_train,
                epochs = 1,
                batch_size=16,
                validation_data =(x_valid, y_valid),
                shuffle = True)
    scores = model.evaluate(x_test, y_test, verbose = 1)
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])
    ret = model.predict(x_test)
    result = []
    pred = []
    for a,b in zip(ret, y_test):
        pred+=[a[0]]
        result+=[b]
    print(pred)
    print(result)
    precision = precision_score(result, pred, average='weighted')
    recall = recall_score(result, pred, average='weighted')
     # calculates f1 for 1:100 dataset with 95tp, 5fn, 55fp
    fmeasure = f1_score(result, pred, average='weighted')
    print('Precision: ', precision)
    print('Recall: ', recall)
    print('F1 Score: ', fmeasure)
    #precisions += [precision]
    #recalls += [recall]
    if scores[1] > 0.5:
        model.save('saved_models/skinSequentialV'+str(pz)+'.h5')
    return [scores[1], precision, recall, fmeasure]

In [39]:
accs=[]
precisions = []
recalls = []
fmeasures = []
for i in range(10):
    ret = main(i)
    accs+=[ret[0]]
    precisions+=[ret[1]]
    recalls+=[ret[2]]
    fmeasures+=[ret[3]]
print(accs)
print(precisions)
print(recalls)
print(fmeasures)

1/1 [==============================] - 0s 310ms/step - loss: 0.6000 - accuracy: 0.4000
Test loss: 0.6000000238418579
Test accuracy: 0.4000000059604645
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0])]
Precision:  0.16000000000000003
Recall:  0.4
F1 Score:  0.2285714285714286


c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 0s 352ms/step - loss: 0.7333 - accuracy: 0.2667
Test loss: 0.7333333492279053
Test accuracy: 0.2666666805744171
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[array([1]), array([1]), array([1]), array([1]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0])]
Precision:  0.07111111111111111
Recall:  0.26666666666666666
F1 Score:  0.11228070175438597


c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 0s 360ms/step - loss: 0.7333 - accuracy: 0.2667
Test loss: 0.7333333492279053
Test accuracy: 0.2666666805744171
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[array([1]), array([1]), array([1]), array([1]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0])]
Precision:  0.07111111111111111
Recall:  0.26666666666666666
F1 Score:  0.11228070175438597


...

c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 0s 399ms/step - loss: 0.6667 - accuracy: 0.3333
Test loss: 0.6666666865348816
Test accuracy: 0.3333333432674408
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[array([1]), array([1]), array([1]), array([1]), array([1]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0])]
Precision:  0.1111111111111111
Recall:  0.3333333333333333
F1 Score:  0.16666666666666666


c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 0s 402ms/step - loss: 0.6000 - accuracy: 0.4000
Test loss: 0.6000000238418579
Test accuracy: 0.4000000059604645
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0])]
Precision:  0.16000000000000003
Recall:  0.4
F1 Score:  0.2285714285714286
[0.4000000059604645, 0.2666666805744171, 0.2666666805744171, 0.4000000059604645, 0.5333333611488342, 0.4000000059604645, 0.3333333432674408, 0.4000000059604645, 0.3333333432674408, 0.4000000059604645]
[0.16000000000000003, 0.07111111111111111, 0.07111111111111111, 0.16000000000000003, 0.28444444444444444, 0.16000000000000003, 0.1111111111111111, 0.16000000000000003, 0.1111111111111111, 0.16000000000000003]
[0.4, 0.26666666666666666, 0.26666666666666666, 0.4, 0.5333333333333333, 0.4, 0.3333333333333333, 0.4, 0.33

c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(len(accs))
print('accs: ', accs)
print('precisions: ', precisions)
print('recalls: ', recalls)
print('fmeasures: ', fmeasures)

In [ ]:
import numpy as np
import scipy.stats as st

#define sample data
data = accs

#create 95% confidence interval for population mean weight
st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))

In [13]:
import numpy as np
import scipy.stats as st

#define sample data
data = [0.8088, 0.600000223, 0.70000]

#create 95% confidence interval for population mean weight
st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))

(0.4435129382506582, 0.9623538770826754)

In [1]:
accs=  [0.7333333492279053, 0.2666666805744171, 0.6666666865348816, 0.4000000059604645, 0.2666666805744171, 0.3333333432674408, 0.5333333611488342, 0.4000000059604645, 0.5333333611488342, 0.4000000059604645]
precisions=  [0.5377777777777778, 0.07111111111111111, 0.4444444444444444, 0.16000000000000003, 0.07111111111111111, 0.1111111111111111, 0.28444444444444444, 0.16000000000000003, 0.28444444444444444, 0.16000000000000003]
recalls=  [0.7333333333333333, 0.26666666666666666, 0.6666666666666666, 0.4, 0.26666666666666666, 0.3333333333333333, 0.5333333333333333, 0.4, 0.5333333333333333, 0.4]
fmeasures=  [0.6205128205128204, 0.11228070175438597, 0.5333333333333333, 0.2285714285714286, 0.11228070175438597, 0.16666666666666666, 0.37101449275362325, 0.2285714285714286, 0.37101449275362325, 0.2285714285714286]

In [2]:
import numpy as np
import scipy.stats as st

#define sample data
data = accs

#create 95% confidence interval for population mean weight
print(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))

data = precisions

#create 95% confidence interval for population mean weight
print(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))

data = recalls

#create 95% confidence interval for population mean weight
print(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))

data = fmeasures

#create 95% confidence interval for population mean weight
print(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))

(0.33914149456038284, 0.567525201511242)
(0.11509963382924633, 0.3417892550596426)
(0.3391414827466517, 0.5675251839200149)
(0.1732072213665765, 0.4213562776820484)
